In [57]:
import tensorflow as tf 
from keras.preprocessing import sequence
import keras 
import os
import numpy as np

In [58]:
path_to_file = "./data/ToolLyrics.txt"
text = open(path_to_file, 'rb').read().decode(encoding='cp1252')
len(text)

96238

In [59]:
#Cleaning un necesary spaces
text = text.replace("\r","\n").replace("\n\n\n\n","").split("\n")
import re

cleaned_text = "\n"
for line in text:
    if line.startswith("[") and line.endswith("]"):
        line = ""
        
text = cleaned_text.join(text)
text = text + "\n" + text #I experimented by doubleing the training data in this way and the models archived better results faster.
len(text)

186149

In [60]:
vocab = sorted(set(text))

char2idx = {u:i for i,u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
    return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [61]:
# print(f'Text: {text[:200]}\nEncoded: {text_as_int[:200]}')

In [62]:
def int_to_text(ints):
    try:
        ints = ints.numpy()
    except:
        print('Already a numpy array')
    return ''.join(idx2char[ints])

# print(int_to_text(text_as_int[:200]))

In [63]:
seq_lenght = 100
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_lenght+1,drop_remainder = True)

In [64]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [94]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)
EMBEDING_DIM = 256
RNN_UNITS = 512

BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [95]:
def build_model(vocab_size, embeding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embeding_dim, batch_input_shape=[batch_size,None]),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

model= build_model(VOCAB_SIZE,EMBEDING_DIM,RNN_UNITS,BATCH_SIZE)

model.summary()

Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_30 (Embedding)    (64, None, 256)           23296     
                                                                 
 lstm_33 (LSTM)              (64, None, 512)           1574912   
                                                                 
 lstm_34 (LSTM)              (64, None, 512)           2099200   
                                                                 
 dense_30 (Dense)            (64, None, 91)            46683     
                                                                 
Total params: 3,744,091
Trainable params: 3,744,091
Non-trainable params: 0
_________________________________________________________________


In [96]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels,logits,from_logits=True)

In [119]:
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True,
    save_freq=1120,
    verbose=1,
)

In [120]:
checkpoint_dir = './checkpoints/lyricsgencheckpoints'

try:
    model= build_model(VOCAB_SIZE,EMBEDING_DIM,RNN_UNITS,64)

    model.load_weights(tf.train.latest_checkpoint(checkpoint_dir)).expect_partial()

#     model.build(tf.TensorShape([1,None]))
    print(f'\nLoaded: {tf.train.latest_checkpoint(checkpoint_dir)}')

except:
    print("\nNo checkpoint to load.")

# model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.000001), loss=loss)
model.compile(optimizer='adam', loss=loss)


Loaded: ./checkpoints/lyricsgencheckpoints\ckpt_40


In [121]:
history = model.fit(data, epochs=40, callbacks = [checkpoint_callback])

Epoch 1/40
28/28 [==============================] - 12s 349ms/step - loss: 0.1451
Epoch 2/40
28/28 [==============================] - 10s 348ms/step - loss: 0.1476
Epoch 3/40
28/28 [==============================] - 7s 247ms/step - loss: 0.1471
Epoch 4/40
28/28 [==============================] - 7s 251ms/step - loss: 0.1457
Epoch 5/40
28/28 [==============================] - 10s 365ms/step - loss: 0.1451
Epoch 6/40
28/28 [==============================] - 10s 354ms/step - loss: 0.1465
Epoch 7/40
28/28 [==============================] - 10s 350ms/step - loss: 0.1449
Epoch 8/40
28/28 [==============================] - 10s 349ms/step - loss: 0.1449
Epoch 9/40
28/28 [==============================] - 10s 349ms/step - loss: 0.1432
Epoch 10/40
28/28 [==============================] - 10s 349ms/step - loss: 0.1415
Epoch 11/40
28/28 [==============================] - 10s 349ms/step - loss: 0.1434
Epoch 12/40
28/28 [==============================] - 10s 353ms/step - loss: 0.1415
Epoch 13/40
28/

In [122]:
model= build_model(VOCAB_SIZE,EMBEDING_DIM,RNN_UNITS,1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir)).expect_partial()

model.build(tf.TensorShape([1,None]))

print(f'{tf.train.latest_checkpoint(checkpoint_dir)}')

./checkpoints/lyricsgencheckpoints\ckpt_40


In [123]:
def generate_text(model, start_string, num_generate=400, temperature=1):
    
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval,0)
    
    text_generated = []
    
    model.reset_states()
    
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions,0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions,num_samples=1)[-1,0].numpy()
        input_eval = tf.expand_dims([predicted_id] , 0)
        
        text_generated.append(idx2char[predicted_id])
        
    return start_string + ''.join(text_generated)

In [ ]:
starting_words = input("Input Starting words")
lyrics_lenght = 1000
temperature = 1
def input_lenght():
    try:
        lyrics_lenght = int(input("Input song lenght, default = 1000"))
    except:
        print("please input a number")
        input_lenght()

def input_temp():
    try:
        lyrics_lenght = float(input("Input temperature, default = 1.0"))
    except:
        print("please input a number")
        input_lenght()
        
input_lenght()
input_temp()

print(f"generateing song with: {starting_words}, {lyrics_lenght}, {temperature}")

In [ ]:
lyric = generate_text(model,starting_words,lyrics_lenght,temperature)
print(lyric)

In [118]:
OUTPUT_FILE_NAME = "test"

with open(f"./outputs/lyrics-generator-outputs/{OUTPUT_FILE_NAME}.txt", "w") as text_file:
    print(f"{lyric}", file=text_file)

In [136]:
int(1.5)

1